In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display
from sklearn.model_selection import train_test_split


In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label


In [ ]:
accuracy = []
accuracy_vsp = []

N_material = 61
N_material_train = 50

for fold in range(10):
    print("=====================")
    
    idx_train = np.array([])
    for i in range(N_material_train):
        train_class = np.random.randint(0, N_material)
        idx_train= np.concatenate((idx_train, np.where(label_material == train_class)[0]))
    
    # need to get better code later
    idx_test = np.setdiff1d(np.arange(N), idx_train)
    
    idx_train = idx_train.astype(int)
    idx_test = idx_test.astype(int)
    
    X_train = raw_feature[idx_train,:]
    Y_VSP_train = label_visiprog[idx_train]
    Y_mat_train = label_material[idx_train]

    X_test = raw_feature[idx_test,:]
    Y_VSP_test = label_visiprog[idx_test]
    Y_mat_test = label_material[idx_test]

    model_vsp = LFDA_VISIPROG()
    model_vsp.fit(X_train, Y_VSP_train)
    X_vsp_test = model_vsp.transform(X_test)
    res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
    accuracy_vsp.append(res_vsp['accuracy'])
    
    
    model = LFDA()
    model.fit(X_train, Y_mat_train)
    X_mat_test = model.transform(X_test)
    res = leave_one_sample_out(X_mat_test, Y_mat_test)
    accuracy.append(res['accuracy'])
    

51
43
19
59
52
50
3
33
33
55
12
49
50
50
51
2
3
12
24
7
2
25
35
49
8
48
15
19
38
57
41
24
60
44
52
1
7
36
15
59
3
42
2
35
9
1
47
33
51
33
59
7
8
28
29
42
38
60
21
27
57
29
50
23
58
26
23
60
44
5
26
5
1
32
4
18
37
21
54
11
4
9
43
1
16
59
21
17
10
22
4
15
40
37
19
44
35
41
59
33
23
51
8
32
5
57
36
19
41
37
4
51
43
22
45
28
10
48
33
47
40
23
18
51
3
16
35
9
41
2
27
38
60
3
18
43
37
43
47
6
22
20
16
30
27
55
32
13
26
47
9
50
18
0
32
44
49
46
47
50
48
44
56
53
35
47
49
32
53
38
18
26
0
9
56
58
24
44
44
49
24
14
18
56
37
55
16
44
23
20
0
14
47
14
42
50
21
28
43
59


In [ ]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy})
display(df)

In [ ]:
np.array(accuracy).mean()

In [ ]:
print(model.transformer())